# day 4

---

## 0. はじめに

ページ上部のメニューバーにある **Kernel** メニューをクリックし、プルダウンメニューから [**Change Kernel ...**] を選び、**gssm2023:Python** を選択してください。

<img src="images/change_kernel1.png" width="30%">

ノートブック上部の右隅に表示されたカーネル名が **gssm2023:Python** になっていることを確認してください。

<img src="images/change_kernel2.png" width="30%">

---

## 1. KHCoder のテキスト解析&分析

### 1.0 事前準備 (関数の定義)

以下のセルを**修正せず**に実行してください

In [ ]:
# 共起ネットワークを描画する
def plot_cooccur_network(df, word_counts, cutoff):

    import networkx as nx
    from networkx.algorithms import community
    import matplotlib.pyplot as plt
    import japanize_matplotlib
    from networkx.drawing.nx_agraph import graphviz_layout
    %matplotlib inline

    Xc = df.values
    Xc_max = Xc.max()

    words = df.columns
    count_max = word_counts.max()

    weights_w, weights_c = [], []
    for i, j in zip(*Xc.nonzero()):
        if i < j and Xc[i,j] > cutoff:
            weights_w.append((words[i], {'weight': word_counts[i] / count_max}))
            weights_w.append((words[j], {'weight': word_counts[j] / count_max}))
            weights_c.append((words[i], words[j], Xc[i,j] / Xc_max))

    G = nx.Graph()
    G.add_nodes_from(weights_w)
    G.add_weighted_edges_from(weights_c)
    G.remove_nodes_from(list(nx.isolates(G)))
    # G = nx.minimum_spanning_tree(G)

    plt.figure(figsize=(10, 10))
    # pos = nx.spring_layout(G, k=0.3)
    pos = graphviz_layout(G, prog='neato', args='-Goverlap="scalexy" -Gsep="+6" -Gnodesep=0.8 -Gsplines="polyline" -GpackMode="graph" -Gstart={}'.format(43))
    weights_n = np.array(list(nx.get_node_attributes(G, 'weight').values()))
    weights_e = np.array(list(nx.get_edge_attributes(G, 'weight').values()))

    communities = community.greedy_modularity_communities(G)
    color_map = []
    for node in G:
        for i, c in enumerate(communities):
            if node in c:
                color_map.append(i)

    nx.draw_networkx_nodes(G, pos, node_color=color_map, alpha=0.7, cmap=plt.cm.Set2, node_size=5000 * weights_n)
    nx.draw_networkx_edges(G, pos, edge_color='gray', edge_cmap=plt.cm.Blues, alpha=0.7, width=3 * weights_e)
    nx.draw_networkx_labels(G, pos, font_family='IPAexGothic')

    plt.axis("off")
    plt.show()


# 共起頻度行列を Jaccard 係数行列に変換する
def jaccard_coef(cooccur_df):

    Xc = cooccur_df.values
    Xj = np.zeros(Xc.shape)
    Xc_sum = cross_df.sum(axis=0).values

    for i, j in zip(*Xc.nonzero()):
        if i < j:
            Xj[i,j] = Xc[i,j] / (Xc[i,j] + Xc_sum[i] + Xc_sum[j])

    jaccard_df = pd.DataFrame(Xj, columns=cooccur_df.columns, index=cooccur_df.columns)

    return jaccard_df


# 対応分析の結果をプロットする
def plot_mca(df):

    import mca
    import numpy as np

    ncols = df.shape[1]
    mca_ben = mca.MCA(df, ncols=ncols, benzecri=False)
    row_coord = mca_ben.fs_r(N=2)
    col_coord = mca_ben.fs_c(N=2)

    eigenvalues = mca_ben.L
    total = np.sum(eigenvalues)
    explained_inertia = eigenvalues / total

    import matplotlib.pyplot as plt
    import japanize_matplotlib
    %matplotlib inline

    plt.figure()

    # Plot of rows (外部変数)
    plt.plot(row_coord[:, 0], row_coord[:, 1], "*", color='red', alpha=0.5)
    for i, label in enumerate(df.index):
        plt.text(row_coord[i, 0], row_coord[i, 1], label, color='red', ha='left', va='bottom')

    # Plot of columns (単語)
    plt.plot(col_coord[:, 0], col_coord[:, 1], "o", color='blue', alpha=0.5)
    for i, label in enumerate(df.columns):
        plt.text(col_coord[i, 0], col_coord[i, 1], label, color='blue', ha='left', va='bottom')

    plt.axvline(0, linestyle='dashed', color='gray', alpha=0.5)
    plt.axhline(0, linestyle='dashed', color='gray', alpha=0.5)
    plt.xlabel(f"Dim 1 ({explained_inertia[0]*100:.3f}%)")
    plt.ylabel(f"Dim 2 ({explained_inertia[1]*100:.3f}%)")
    # plt.axis('equal')
    plt.show()

### 1.1 データのダウンロード (前回ダウンロード済みのためスキップ)

以下のデータがダウンロード済みです

| ファイル名 | 件数 | データセット | 備考 |
| --- | --- | --- | --- |
| rakuten-1000-2022-2023.xlsx.zip | 10,000 | •レジャー+ビジネスの 10エリア<br>•エリアごと 1,000件 (ランダムサンプリング)<br>•期間: 2022/1~2023 GW明け | 本講義の全体を通して使用する |

In [ ]:
# もし、再度ダウンロードが必要な場合は残りの行のコメントマーク「#」を除去して、このセルを再実行してください

# FILE_ID = "1n-uvGoH7XQhxexN57hYXuFrkGeHKp-HV"
# !gdown --id {FILE_ID}
# !unzip rakuten-1000-2022-2023.xlsx.zip

### 1.2 データの読み込み (DataFrame型)

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_excel("rakuten-1000-2022-2023.xlsx")
print(df.shape)
display(df.head())

### 1.3 「文書-抽出語」表 を作成する

コメント列から単語を抽出する (単語を品詞「名詞」「形容詞」「未知語」で絞り込む)

In [ ]:
from collections import defaultdict
import MeCab

tagger = MeCab.Tagger("-r ../tools/usr/local/etc/mecabrc --unk-feature 未知語")

word_counts = defaultdict(lambda: 1)
words = []

ZEN = "".join(chr(0xff01 + i) for i in range(94))
HAN = "".join(chr(0x21 + i) for i in range(94))
HAN2ZEN = str.maketrans(HAN, ZEN)

# stopwords = ['する', 'ある', 'ない', 'いう', 'もの', 'こと', 'よう', 'なる', 'ほう']
stopwords = ["湯畑"]

for index, row in df.iterrows():
    node = tagger.parseToNode(row["コメント"].translate(HAN2ZEN))
    while node:
        features = node.feature.split(',')
        pos1 = features[0]
        pos2 = features[1] if len(features) > 1 else ""
        base = features[6] if len(features) > 6 else None

        if base not in stopwords:

            if (pos1 == "名詞" and pos2 == "一般"):
                base = base if base is not None else node.surface
                postag = "名詞"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            elif (pos1 == "名詞" and pos2 == "形容動詞語幹"):
                base = base if base is not None else node.surface
                base = f"{base}だ"
                postag = "形容動詞"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            elif pos1 == "形容詞":
                base = base if base is not None else node.surface
                postag = "形容詞"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

            elif pos1 == "未知語":
                base = base if base is not None else node.surface
                postag = "未知語"
                key = (base, postag)
                word_counts[key] += 1
                words.append([index + 1, base, postag, row["カテゴリー"], row["エリア"], key])

        node = node.next

columns = [
    "文書ID",
    # "単語ID",
    "表層",
    "品詞",
    "カテゴリー",
    "エリア",
    "dict_key",
]
docs_df = pd.DataFrame(words, columns=columns)
print(docs_df.shape)
display(docs_df.head())

抽出語の出現頻度をカウントする

In [ ]:
word_list = []
for i, (k, v) in enumerate(sorted(word_counts.items(), key=lambda x:x[1], reverse=True)):
    word_list.append((i, k[0], v, k))

columns = [
    "単語ID",
    "表層",
    "出現頻度",
    "dict_key"
]
word_counts_df = pd.DataFrame(word_list, columns=columns)[0:75]
print(word_counts_df.shape)
display(word_counts_df.head(10))

単語IDを紐つける

In [ ]:
merged_df = pd.merge(docs_df, word_counts_df, how="inner", on="dict_key", suffixes=["", "_right"])
docs_df = merged_df[["文書ID", "単語ID", "表層", "品詞", "カテゴリー", "エリア", "dict_key"]]
print(docs_df.shape)
display(docs_df)

「文書-抽出語」表 を作成する

In [ ]:
cross_df = pd.crosstab([docs_df['カテゴリー'], docs_df['エリア'], docs_df['文書ID']], docs_df['単語ID'], margins=False)
cross_df.columns = word_counts_df["表層"]
print(cross_df.shape)
display(cross_df)

### 1.4 共起ネットワーク

共起度行列を作成する

In [ ]:
from scipy.sparse import csc_matrix

X = csc_matrix(cross_df.values)
X[X > 0] = 1
Xc = (X.T * X)
Xc = np.triu(Xc.toarray())

cooccur_df = pd.DataFrame(Xc, columns=cross_df.columns, index=cross_df.columns)
print(cooccur_df.shape)
display(cooccur_df.head())

#### (a) 共起ネットワーク (共起度の高いエッジを残す)

In [ ]:
word_counts = cross_df.sum(axis=0).values
plot_cooccur_network(cooccur_df, word_counts, cooccur_df.values.max() * 0.05)

#### (b) 共起ネットワーク (Jaccard 係数が上位のエッジを残す)

In [ ]:
jaccard_df = jaccard_coef(cooccur_df)

word_counts = cross_df.sum(axis=0).values
plot_cooccur_network(jaccard_df, word_counts, np.sort(jaccard_df.values.reshape(-1))[::-1][60])

### 1.5 対応分析

「文書-抽出語」 表を確認する

In [ ]:
print(cross_df.shape)
display(cross_df.head())

「外部変数-抽出語」 クロス集計表を作成する

In [ ]:
category_cross_df = cross_df.groupby(level='カテゴリー').sum()
area_cross_df = cross_df.groupby(level='エリア').sum()
aggregate_df = pd.concat([category_cross_df, area_cross_df])
print(aggregate_df.shape)
display(aggregate_df)

#### (a) 対応分析プロット (ライブラリ mca を使用)

In [ ]:
plot_mca(aggregate_df)

#### (b) 対応分析プロット (カイ2乗値を手計算してプロットする)

In [ ]:
def plot_coresp(row_coord, col_coord, row_labels, col_labels, explained_inertia=None):

    import matplotlib.pyplot as plt
    import japanize_matplotlib
    %matplotlib inline

    plt.figure()

    # Plot of rows (外部変数)
    plt.plot(row_coord[:, 0], row_coord[:, 1], "*", color='red', alpha=0.5)
    for i, label in enumerate(row_labels):
        plt.text(row_coord[i, 0], row_coord[i, 1], label, color='red', ha='left', va='bottom')

    # Plot of columns (単語)
    plt.plot(col_coord[:, 0], col_coord[:, 1], "o", color='blue', alpha=0.5)
    for i, label in enumerate(col_labels):
        plt.text(col_coord[i, 0], col_coord[i, 1], label, color='blue', ha='left', va='bottom')

    plt.axvline(0, linestyle='dashed', color='gray', alpha=0.5)
    plt.axhline(0, linestyle='dashed', color='gray', alpha=0.5)

    if explained_inertia is not None:
        plt.xlabel(f"Dim 1 ({explained_inertia[0]:.3f}%)")
        plt.ylabel(f"Dim 2 ({explained_inertia[1]:.3f}%)")

    # plt.axis('equal')
    plt.show()


In [ ]:
import numpy as np

table_N = aggregate_df.values
row_sum = table_N.sum(axis=1)
col_sum = table_N.sum(axis=0)
n = aggregate_df.values.sum()

expected = np.outer(row_sum, col_sum) / n
chisq = np.square(table_N - expected) / expected
residuals = (table_N - expected) / np.sqrt(expected)

# Standardized residuals
residuals = residuals / np.sqrt(n)

# Number of dimensions
nb_axes = min(residuals.shape[0]-1, residuals.shape[1]-1)

# Singular value decomposition
U, s, V = np.linalg.svd(residuals, full_matrices=True)
sv = s[:nb_axes]
u = U[:, :nb_axes]
v = V[:nb_axes, :]

# row mass
row_mass = row_sum / n

# row coord = u * sv /sqrt(row.mass)
row_coord = (u * sv) / np.sqrt(row_mass)[:, np.newaxis]

# col mass
col_mass = col_sum / n

# row coord = sv * vT /sqrt(col.mass)
col_coord = (sv * v.T) / np.sqrt(col_mass)[:, np.newaxis]

# eige nvalue
eige_nvalue = s ** 2

# contribution rate 
explained_inertia = 100 * eige_nvalue / sum(eige_nvalue)

row_labels = aggregate_df.index
col_labels = aggregate_df.columns
plot_coresp(row_coord, col_coord,row_labels, col_labels, explained_inertia)

#### (c) 対応分析プロット (共起頻度のままプロットする)

In [ ]:
import numpy as np

table_N = aggregate_df.values
table_P = table_N / aggregate_df.values.max()

# Singular value decomposition
U, s, V = np.linalg.svd(table_P, full_matrices=True)
sv = s[:nb_axes]
u = U[:, :nb_axes]
v = V[:nb_axes, :]

# row mass
row_mass = row_sum / n

# row coord = u * sv /sqrt(row.mass)
row_coord = (u * sv) / np.sqrt(row_mass)[:, np.newaxis]

# col mass
col_mass = col_sum / n

# row coord = sv * vT /sqrt(col.mass)
col_coord = (sv * v.T) / np.sqrt(col_mass)[:, np.newaxis]

# eige nvalue
eige_nvalue = s ** 2

# contribution rate 
explained_inertia = 100 * eige_nvalue / sum(eige_nvalue)

row_labels = aggregate_df.index
col_labels = aggregate_df.columns
plot_coresp(row_coord, col_coord,row_labels, col_labels, explained_inertia)

#### (d) PCAプロット (共起頻度のままプロットする)

In [ ]:
def plot_pca(coeff, reduced, row_labels, col_labels, var_ratio=None):

    import matplotlib.pyplot as plt
    import japanize_matplotlib
    %matplotlib inline

    plt.figure()

    # Plot of rows (外部変数)
    for i, label in enumerate(row_labels):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], color='r', alpha=0.5)
        plt.text(coeff[i, 0], coeff[i, 1], label, color='red', ha='left', va='bottom')

    # Plot of columns (単語)
    plt.plot(reduced[:, 0], reduced[:, 1], "o", color='blue', alpha=0.5)
    for i, label in enumerate(col_labels):
        plt.text(reduced[i, 0], reduced[i, 1], label, color='blue', ha='left', va='bottom')

    plt.axvline(0, linestyle='dashed', color='gray', alpha=0.5)
    plt.axhline(0, linestyle='dashed', color='gray', alpha=0.5)

    if var_ratio is not None:
        plt.xlabel(f"Dim 1 ({var_ratio[0]*100:.3f}%)")
        plt.ylabel(f"Dim 2 ({var_ratio[1]*100:.3f}%)")

    # plt.axis('equal')
    plt.show()


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

table_N = aggregate_df.values

pca = PCA()

reduced = pca.fit_transform(table_N.T)
coeff = np.transpose(pca.components_)
var_ratio = pca.explained_variance_ratio_

scalex = 1.0 / (reduced[:,0].max() - reduced[:,0].min())
scaley = 1.0 / (reduced[:,1].max() - reduced[:,1].min())
reduced[:,0] *= scalex
reduced[:,1] *= scaley

plot_pca(coeff, reduced, row_labels, col_labels, var_ratio)

#### (e) PCAプロット (カイ2乗値をプロットする)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import japanize_matplotlib
%matplotlib inline

import numpy as np

table_N = aggregate_df.values
row_sum = table_N.sum(axis=1)
col_sum = table_N.sum(axis=0)
n = aggregate_df.values.sum()

expected = np.outer(row_sum, col_sum) / n
chisq = np.square(table_N - expected) / expected
residuals = (table_N - expected) / np.sqrt(expected)

# Standardized residuals
residuals = residuals / np.sqrt(n)

pca = PCA()

reduced = pca.fit_transform(residuals.T)
coeff = np.transpose(pca.components_)
var_ratio = pca.explained_variance_ratio_

scalex = 1.0 / (reduced[:,0].max() - reduced[:,0].min())
scaley = 1.0 / (reduced[:,1].max() - reduced[:,1].min())
reduced[:,0] *= scalex
reduced[:,1] *= scaley

plot_pca(coeff, reduced, row_labels, col_labels, var_ratio)

---

## 2. テキスト分析 (実践編)

### 2.1 カテゴリーやエリアごとのユーザーの注目ポイントを押さえる